### This script will set up, simulate and gather all results for all cell rotations, for different model and/or field instantiations. The available settings are: 

**A:** Passive models with axons. Subthreshold simulation. Plane field 2.5 mm away from soma. <br />
**B:** Passive models without axons. Subthreshold simulation. Plane field 2.5 mm away from soma. <br />

**C:** Active models without axons. Subthreshold simulation. Plane field 2.5 mm away from soma. <br />
**D:** Active models without axons. Subthreshold simulation. Point field 50 um away from soma. <br />
**E:** Passive models without axons. Subthreshold simulation. Point field 50 um away from soma. <br />

**F:** Active models without axons. Suprathreshold simulation with calibrated DC injection. Plane field 2.5 mm away from soma. <br />
**G:** Active models without axons. Suprathreshold simulation with calibrated DC injection. Point field 50 um away from soma. <br />

**H:** Active models without axons. Suprathreshold simulation without ES. <br />

**I:** Active models with custom passive axons with Cm = A. Suprathreshold simulation without ES. <br />
**J:** Active models with custom passive axons with Cm = A/4. Suprathreshold simulation without ES. <br />
**K:** Active models with custom passive axons with Cm = A/8. Suprathreshold simulation without ES. <br />

**L:** Active models with custom active axons with dendrite parameters. Suprathreshold simulation without ES. <br />
**M:** Active models with custom active axons with soma parameters. Suprathreshold simulation without ES. <br />

**N:** Active models with custom active axons with dendrite parameters all HoF. Suprathreshold simulation without ES. <br />

In [1]:
Selected_Sweep = 'N'

simulations_in_parallel = 26
model_processing = 'csmc_allactive'

if Selected_Sweep == 'A':
    model_file = '../Required_Files/Models/JSON Format/A.json'
    swc_path = '../Required_Files/Models/SWCs connected With/'
    supra = False
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = False
    Custom = False
    allHOF = False
    Sub_amplitudes = [1,3,5,10,30,50,100,300,500,1000] # uA
    Sub_frequencies =[1,3,5,8,10,28,30,50,100,140,300] # Hz
elif Selected_Sweep == 'B':
    model_file = '../Required_Files/Models/JSON Format/A.json'
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = False
    Custom = False
    allHOF = False
    Sub_amplitudes = [1,3,5,10,30,50,100,300,500,1000] # uA
    Sub_frequencies =[1,3,5,8,10,28,30,50,100,140,300] # Hz
elif Selected_Sweep == 'C':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = False
    Custom = False
    allHOF = False
    Sub_amplitudes = [10,100,500] # uA
    Sub_frequencies =[1,3,5,8,10,28,30,50,100,140,300] # Hz
elif Selected_Sweep == 'D':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = True
    Rotation_Steps = 10 # deg
    Only_Calibrated = False
    Custom = False
    allHOF = False
    Sub_amplitudes = [10,100,500] # uA
    Sub_frequencies =[1,8,28,140]
elif Selected_Sweep == 'E':
    model_file = '../Required_Files/Models/JSON Format/A.json'
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = False
    point = True
    Rotation_Steps = 10 # deg
    Only_Calibrated = False
    Custom = False
    allHOF = False
    Sub_amplitudes = [10,100,500] # uA
    Sub_frequencies =[1,8,28,140]
elif Selected_Sweep == 'F':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = True
    point = False
    Rotation_Steps = 30 # deg
    Only_Calibrated = True
    Custom = False
    allHOF = False
    SuP_amplitudes = [0.0,10,100,500] # uA
    SuP_frequencies =[8,28,140] # Hz
elif Selected_Sweep == 'G':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = True
    point = True
    Rotation_Steps = 10 # deg
    Only_Calibrated = True
    Custom = False
    allHOF = False
    SuP_amplitudes_P = [0.0,0.05,0.1,0.2] # uA
    SuP_frequencies =[8,28,140] # Hz
elif Selected_Sweep == 'H':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Without/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = None
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'I':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = 'A'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'J':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = 'A/4'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'K':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = 'A/8'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'L':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = 'D'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'M':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = False
    axon_parameters = 'S'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz
elif Selected_Sweep == 'N':
    model_file = False
    swc_path = '../Required_Files/Models/SWCs connected Custom/'
    supra = True
    point = False
    Rotation_Steps = 360 # deg
    Only_Calibrated = False
    Custom = True
    allHOF = True
    axon_parameters = 'D'
    SuP_amplitudes = [0] # uA
    SuP_frequencies =[1] # Hz

##### Set up directories

In [2]:
import shutil, os, json
import pandas as pd

morphfiles = os.listdir(swc_path)

inhpath = '../Required_files/Models/JSON Format/Inh Models/'
excpath = '../Required_files/Models/JSON Format/Exc Models/'
inhtypes = os.listdir(inhpath)
exctypes = os.listdir(excpath)

allmodels = []
for i in inhtypes:
    for j in os.listdir(inhpath + i):
        allmodels.append(inhpath + i + '/' + j + '/' + j + '_stage_1_hof_0.json')

for i in exctypes:
    for j in os.listdir(excpath + i):
        allmodels.append(excpath + i + '/' + j + '/' + j + '_stage_1_hof_0.json')

current_vals = pd.read_csv('../Required_files/Calibrated.csv')
current_vals = current_vals.sort_values('Target', ascending=True).reset_index(drop=True).reset_index()
current_vals = current_vals.set_index('cell_id')

simulation_no = 0
for i in morphfiles:
    if int(i.split('rmed_')[1].split('.sw')[0]) not in range(0,360,Rotation_Steps):
        continue
    if int(i.split('_')[2]) not in current_vals.index and Only_Calibrated:
        continue
    if supra:
        frequencies = SuP_frequencies
    else:
        frequencies = Sub_frequencies
    for f in frequencies:
        if supra:
            if point:
                amplitudes = SuP_amplitudes_P
            else:
                amplitudes = SuP_amplitudes
        else:
            amplitudes = Sub_amplitudes
        for c in amplitudes:
            if allHOF:
                hof_range = 20
            else:
                hof_range = 1
            for hof in range(hof_range):
                full_path = os.getcwd().replace('\\','/') + '/Simulations/' + Selected_Sweep + '/SIM_' + \
                str(simulation_no)
                if os.path.exists(full_path):
                    if os.path.isfile(full_path + '/Data.csv'):
                        simulation_no += 1
                        continue
                    else:
                        shutil.rmtree(full_path)
                os.makedirs(full_path)
                shutil.copyfile(swc_path + i, full_path + '/morph.swc')
                shutil.copyfile('../Required_Files/Simulate.py', full_path + '/simulate.py')
                shutil.copyfile('../Required_Files/cell_functions.py', full_path + '/cell_functions.py')
                shutil.copyfile('../Required_Files/nrnmech.dll', full_path + '/nrnmech.dll')
                if model_file:
                    shutil.copyfile(model_file, full_path + '/model.json')
                else:
                    for mod in allmodels:
                        if i.split("transformed")[0] in mod:
                            inpa = mod.replace('hof_0','hof_'+str(hof))
                            oupa = full_path + '/model.json'
                            with open(inpa, 'r') as infile:
                                parameters = json.load(infile)
                            del parameters['axon_morph']
                            if Custom and ('Without' not in swc_path):
                                parameters['conditions'][0]['erev'].append(parameters['conditions'][0]['erev'][0])
                                parameters['conditions'][0]['erev'][-1]['section'] = 'axon'
                                if 'A' in axon_parameters:   
                                    with open('../Required_Files/Models/JSON Format/A.json', 'r') as infile:
                                        passive_param = json.load(infile)
                                    axonlist = []
                                    gendict = passive_param['genome']
                                    for di in gendict:
                                        if di['section']=='soma':
                                            if '/' in axon_parameters:
                                                di['value'] = di['value']/int(axon_parameters.split('/')[1])
                                            elif '*' in axon_parameters:
                                                di['value'] = di['value']*int(axon_parameters.split('*')[1])
                                            axonlist.append(di.copy())
                                    for gi in range(len(axonlist)):
                                        axonlist[gi]['section']='axon'
                                        parameters['genome'].append(axonlist[gi])
                                elif 'S' in axon_parameters:
                                    axonlist = []
                                    gendict = parameters['genome']
                                    for di in gendict:
                                        if di['section']=='soma':
                                            axonlist.append(di.copy())
                                    for gi in range(len(axonlist)):
                                        axonlist[gi]['section']='axon'
                                        parameters['genome'].append(axonlist[gi])
                                elif 'D' in axon_parameters:
                                    axonlist = []
                                    gendict = parameters['genome']
                                    for di in gendict:
                                        if di['section']=='dend':
                                            axonlist.append(di.copy())
                                    for gi in range(len(axonlist)):
                                        axonlist[gi]['section']='axon'
                                        parameters['genome'].append(axonlist[gi])
                            elif ('Without' not in swc_path):
                                parameters['conditions'][0]['erev'].append(parameters['conditions'][0]['erev'][0])
                                parameters['conditions'][0]['erev'][-1]['section'] = 'axon'
                                axonlist = []
                                gendict = parameters['genome']
                                for di in gendict:
                                    if di['section']=='dend':
                                        axonlist.append(di.copy())
                                for gi in range(len(axonlist)):
                                    axonlist[gi]['section']='axon'
                                    gendict.append(axonlist[gi])
                                parameters['genome'] = gendict
                            json_object = json.dumps(parameters, indent=4)
                            with open(oupa, 'w') as outfile:
                                outfile.write(json_object)
                            break
                if Custom:
                    inj_curr = 0.3
                    inj_std = 0.0
                    exp_rate = 0
                elif supra:
                    inj_curr = current_vals.loc[int(i.split('_')[2]),'Mean']
                    inj_std = current_vals.loc[int(i.split('_')[2]),'StD']
                    exp_rate = current_vals.loc[int(i.split('_')[2]),'PM']/2
                else:
                    inj_curr = 0
                    inj_std = 0
                    exp_rate = 200
                fin = open(full_path + '/simulate.py', "rt")
                cnt = 10
                lines = []
                for line in fin:
                    if cnt==10:
                        lines.append('base_dir = "' + full_path + '"\n')
                    elif cnt==9:
                        lines.append('inj_curr = ' + str(inj_curr) + '\n')
                    elif cnt==8:
                        lines.append('inj_std = ' + str(inj_std) + '\n')
                    elif cnt==7:
                        lines.append('exp_rate = ' + str(exp_rate) + '\n')
                    elif cnt==6:
                        lines.append('es_freq = ' + str(f) + '\n')
                    elif cnt==5:
                        lines.append('es_ampl = ' + str(c) + '\n')
                    elif cnt==4:
                        lines.append('mod_proc = "' + model_processing + '"\n')
                    elif cnt==3:
                        if point:
                            lines.append('point = 1\n')
                        else:
                            lines.append('point = 0\n')
                    elif cnt==2:
                        lines.append('run = 1\n')
                    elif cnt==1:
                        if Custom:
                            if axon_parameters is not None:
                                lines.append('returns = 3\n')
                            else:
                                lines.append('returns = 4\n')
                        elif supra:
                            lines.append('returns = 1\n')
                        else:
                            lines.append('returns = 0\n')
                    else:
                        lines.append(line)
                    cnt -= 1
                fin.close()
                fout = open(full_path + '/simulate.py', "wt")
                for line in lines:
                    fout.write(line)
                fout.close()

                simulation_no += 1
print (simulation_no)

4260


##### Simulate

In [3]:
import subprocess as sbp
import os, shutil, time

total_simulations = simulation_no
print(total_simulations)
processes = []
process_names = []
active_count = 0
process_index = 0

start_time = time.time()
while (active_count > 0) or (process_index < total_simulations):
    active_processes = processes.copy()
    active_process_names = process_names.copy()
    processes = []
    process_names = []
    for i, sbprc in enumerate(active_processes):
        poll = sbprc.poll()
        if poll is None:
            processes.append(sbprc)
            process_names.append(active_process_names[i])
        else:
            active_count -= 1
            print ('Finished ' + active_process_names[i])
            if process_index >= total_simulations:
                print ('Active: ' + str(active_count) + ', Pending: ' + str(total_simulations-process_index))
    if (active_count < simulations_in_parallel) and (process_index < total_simulations):
        process_name = 'SIM_'+str(process_index)
        if not os.path.isfile(os.getcwd().replace('\\','/') + '/SIMULATIONS/' + Selected_Sweep + \
                              '/' + process_name + "/Data.csv"):
            cmnd = ["python", 
                    os.getcwd().replace('\\','/') + '/SIMULATIONS/' + Selected_Sweep + '/' +  \
                    process_name + "/simulate.py"]
            p = sbp.Popen(cmnd)
            processes.append(p)
            process_names.append(process_name)
            active_count += 1
        process_index += 1
        print ('Active: ' + str(active_count) + ', Pending: ' + str(total_simulations-process_index))
    if active_count >= simulations_in_parallel:
        time.sleep(2)
    
queue_time = time.time() - start_time
print("Total time " + str(queue_time/3600) + " hours")

4260
Active: 1, Pending: 4259
Active: 2, Pending: 4258
Active: 3, Pending: 4257
Active: 4, Pending: 4256
Active: 5, Pending: 4255
Active: 6, Pending: 4254
Active: 7, Pending: 4253
Active: 8, Pending: 4252
Active: 9, Pending: 4251
Active: 10, Pending: 4250
Active: 11, Pending: 4249
Active: 12, Pending: 4248
Active: 13, Pending: 4247
Active: 14, Pending: 4246
Active: 15, Pending: 4245
Active: 16, Pending: 4244
Active: 17, Pending: 4243
Active: 18, Pending: 4242
Active: 19, Pending: 4241
Active: 20, Pending: 4240
Active: 21, Pending: 4239
Active: 22, Pending: 4238
Active: 23, Pending: 4237
Active: 24, Pending: 4236
Active: 25, Pending: 4235
Active: 26, Pending: 4234
Finished SIM_20
Finished SIM_21
Finished SIM_22
Finished SIM_23
Finished SIM_24
Finished SIM_25
Active: 21, Pending: 4233
Active: 22, Pending: 4232
Active: 23, Pending: 4231
Active: 24, Pending: 4230
Active: 25, Pending: 4229
Active: 26, Pending: 4228
Finished SIM_1
Finished SIM_2
Finished SIM_3
Finished SIM_4
Finished SIM_6
F

Finished SIM_194
Finished SIM_195
Finished SIM_196
Finished SIM_197
Finished SIM_198
Active: 22, Pending: 4039
Active: 23, Pending: 4038
Active: 24, Pending: 4037
Active: 25, Pending: 4036
Active: 26, Pending: 4035
Finished SIM_199
Active: 26, Pending: 4034
Finished SIM_200
Finished SIM_201
Finished SIM_202
Finished SIM_203
Finished SIM_204
Finished SIM_205
Finished SIM_206
Finished SIM_207
Finished SIM_208
Finished SIM_209
Finished SIM_210
Finished SIM_211
Finished SIM_212
Finished SIM_213
Finished SIM_214
Finished SIM_215
Finished SIM_216
Finished SIM_217
Finished SIM_218
Finished SIM_219
Active: 7, Pending: 4033
Active: 8, Pending: 4032
Active: 9, Pending: 4031
Active: 10, Pending: 4030
Active: 11, Pending: 4029
Active: 12, Pending: 4028
Active: 13, Pending: 4027
Active: 14, Pending: 4026
Active: 15, Pending: 4025
Active: 16, Pending: 4024
Active: 17, Pending: 4023
Active: 18, Pending: 4022
Active: 19, Pending: 4021
Active: 20, Pending: 4020
Active: 21, Pending: 4019
Active: 22, Pen

Finished SIM_386
Finished SIM_387
Finished SIM_388
Finished SIM_389
Finished SIM_390
Finished SIM_391
Finished SIM_392
Finished SIM_393
Finished SIM_394
Active: 18, Pending: 3847
Active: 19, Pending: 3846
Active: 20, Pending: 3845
Active: 21, Pending: 3844
Active: 22, Pending: 3843
Active: 23, Pending: 3842
Active: 24, Pending: 3841
Active: 25, Pending: 3840
Active: 26, Pending: 3839
Finished SIM_395
Finished SIM_396
Finished SIM_397
Finished SIM_398
Finished SIM_399
Active: 22, Pending: 3838
Active: 23, Pending: 3837
Active: 24, Pending: 3836
Active: 25, Pending: 3835
Active: 26, Pending: 3834
Finished SIM_400
Finished SIM_401
Finished SIM_402
Finished SIM_403
Finished SIM_404
Finished SIM_405
Finished SIM_406
Finished SIM_407
Finished SIM_408
Finished SIM_409
Finished SIM_410
Finished SIM_411
Finished SIM_412
Finished SIM_413
Finished SIM_414
Finished SIM_416
Finished SIM_417
Finished SIM_418
Finished SIM_420
Active: 8, Pending: 3833
Active: 9, Pending: 3832
Active: 10, Pending: 3831

Finished SIM_581
Finished SIM_582
Finished SIM_584
Finished SIM_585
Finished SIM_586
Finished SIM_587
Finished SIM_588
Finished SIM_589
Finished SIM_590
Active: 18, Pending: 3651
Active: 19, Pending: 3650
Active: 20, Pending: 3649
Active: 21, Pending: 3648
Active: 22, Pending: 3647
Active: 23, Pending: 3646
Active: 24, Pending: 3645
Active: 25, Pending: 3644
Active: 26, Pending: 3643
Finished SIM_592
Finished SIM_594
Finished SIM_595
Active: 24, Pending: 3642
Active: 25, Pending: 3641
Active: 26, Pending: 3640
Finished SIM_591
Finished SIM_593
Finished SIM_596
Active: 24, Pending: 3639
Active: 25, Pending: 3638
Active: 26, Pending: 3637
Finished SIM_602
Active: 26, Pending: 3636
Finished SIM_598
Finished SIM_600
Finished SIM_601
Finished SIM_603
Finished SIM_604
Finished SIM_605
Finished SIM_606
Finished SIM_607
Finished SIM_608
Finished SIM_610
Finished SIM_611
Finished SIM_612
Finished SIM_613
Finished SIM_614
Finished SIM_615
Active: 12, Pending: 3635
Active: 13, Pending: 3634
Activ

Finished SIM_775
Finished SIM_776
Finished SIM_777
Finished SIM_778
Finished SIM_779
Active: 22, Pending: 3458
Active: 23, Pending: 3457
Active: 24, Pending: 3456
Active: 25, Pending: 3455
Active: 26, Pending: 3454
Finished SIM_780
Finished SIM_782
Active: 25, Pending: 3453
Active: 26, Pending: 3452
Finished SIM_781
Finished SIM_783
Finished SIM_785
Finished SIM_788
Active: 23, Pending: 3451
Active: 24, Pending: 3450
Active: 25, Pending: 3449
Active: 26, Pending: 3448
Finished SIM_784
Finished SIM_786
Finished SIM_787
Finished SIM_790
Finished SIM_791
Finished SIM_792
Finished SIM_795
Active: 20, Pending: 3447
Active: 21, Pending: 3446
Active: 22, Pending: 3445
Active: 23, Pending: 3444
Active: 24, Pending: 3443
Active: 25, Pending: 3442
Active: 26, Pending: 3441
Finished SIM_789
Finished SIM_793
Finished SIM_794
Finished SIM_796
Finished SIM_798
Finished SIM_799
Finished SIM_800
Active: 20, Pending: 3440
Active: 21, Pending: 3439
Active: 22, Pending: 3438
Active: 23, Pending: 3437
Act

...

Finished SIM_3458
Finished SIM_3459
Finished SIM_3466
Active: 24, Pending: 769
Active: 25, Pending: 768
Active: 26, Pending: 767
Finished SIM_3467
Finished SIM_3468
Finished SIM_3470
Active: 24, Pending: 766
Active: 25, Pending: 765
Active: 26, Pending: 764
Finished SIM_3469
Finished SIM_3471
Finished SIM_3472
Finished SIM_3473
Finished SIM_3476
Active: 22, Pending: 763
Active: 23, Pending: 762
Active: 24, Pending: 761
Active: 25, Pending: 760
Active: 26, Pending: 759
Finished SIM_3474
Finished SIM_3475
Finished SIM_3477
Finished SIM_3480
Active: 23, Pending: 758
Active: 24, Pending: 757
Active: 25, Pending: 756
Active: 26, Pending: 755
Finished SIM_3478
Finished SIM_3481
Finished SIM_3482
Finished SIM_3483
Finished SIM_3489
Active: 22, Pending: 754
Active: 23, Pending: 753
Active: 24, Pending: 752
Active: 25, Pending: 751
Active: 26, Pending: 750
Finished SIM_3479
Finished SIM_3484
Finished SIM_3485
Finished SIM_3486
Finished SIM_3487
Finished SIM_3488
Active: 21, Pending: 749
Active:

Finished SIM_3656
Active: 26, Pending: 577
Finished SIM_3657
Finished SIM_3658
Finished SIM_3660
Finished SIM_3663
Finished SIM_3664
Active: 22, Pending: 576
Active: 23, Pending: 575
Active: 24, Pending: 574
Active: 25, Pending: 573
Active: 26, Pending: 572
Finished SIM_3659
Finished SIM_3661
Finished SIM_3662
Finished SIM_3665
Finished SIM_3666
Finished SIM_3667
Finished SIM_3668
Finished SIM_3669
Finished SIM_3670
Finished SIM_3671
Finished SIM_3672
Finished SIM_3673
Active: 15, Pending: 571
Active: 16, Pending: 570
Active: 17, Pending: 569
Active: 18, Pending: 568
Active: 19, Pending: 567
Active: 20, Pending: 566
Active: 21, Pending: 565
Active: 22, Pending: 564
Active: 23, Pending: 563
Active: 24, Pending: 562
Active: 25, Pending: 561
Active: 26, Pending: 560
Finished SIM_3675
Active: 26, Pending: 559
Finished SIM_3674
Finished SIM_3676
Finished SIM_3677
Active: 24, Pending: 558
Active: 25, Pending: 557
Active: 26, Pending: 556
Finished SIM_3678
Finished SIM_3680
Finished SIM_3681


Finished SIM_3847
Finished SIM_3848
Finished SIM_3849
Finished SIM_3850
Finished SIM_3852
Finished SIM_3853
Finished SIM_3854
Active: 20, Pending: 384
Active: 21, Pending: 383
Active: 22, Pending: 382
Active: 23, Pending: 381
Active: 24, Pending: 380
Active: 25, Pending: 379
Active: 26, Pending: 378
Finished SIM_3856
Finished SIM_3857
Finished SIM_3858
Finished SIM_3859
Finished SIM_3860
Finished SIM_3861
Finished SIM_3862
Active: 20, Pending: 377
Active: 21, Pending: 376
Active: 22, Pending: 375
Active: 23, Pending: 374
Active: 24, Pending: 373
Active: 25, Pending: 372
Active: 26, Pending: 371
Finished SIM_3863
Finished SIM_3864
Finished SIM_3865
Finished SIM_3866
Active: 23, Pending: 370
Active: 24, Pending: 369
Active: 25, Pending: 368
Active: 26, Pending: 367
Finished SIM_3867
Finished SIM_3869
Finished SIM_3870
Active: 24, Pending: 366
Active: 25, Pending: 365
Active: 26, Pending: 364
Finished SIM_3868
Active: 26, Pending: 363
Finished SIM_3871
Finished SIM_3872
Active: 25, Pendin

Finished SIM_4041
Finished SIM_4042
Finished SIM_4045
Finished SIM_4047
Finished SIM_4048
Active: 22, Pending: 190
Active: 23, Pending: 189
Active: 24, Pending: 188
Active: 25, Pending: 187
Active: 26, Pending: 186
Finished SIM_4046
Finished SIM_4049
Active: 25, Pending: 185
Active: 26, Pending: 184
Finished SIM_4050
Finished SIM_4051
Finished SIM_4052
Finished SIM_4053
Finished SIM_4054
Finished SIM_4055
Finished SIM_4056
Active: 20, Pending: 183
Active: 21, Pending: 182
Active: 22, Pending: 181
Active: 23, Pending: 180
Active: 24, Pending: 179
Active: 25, Pending: 178
Active: 26, Pending: 177
Finished SIM_4060
Finished SIM_4061
Active: 25, Pending: 176
Active: 26, Pending: 175
Finished SIM_4062
Finished SIM_4063
Finished SIM_4064
Active: 24, Pending: 174
Active: 25, Pending: 173
Active: 26, Pending: 172
Finished SIM_4057
Finished SIM_4059
Finished SIM_4065
Finished SIM_4066
Finished SIM_4067
Finished SIM_4068
Active: 21, Pending: 171
Active: 22, Pending: 170
Active: 23, Pending: 169


Finished SIM_4246
Active: 20, Pending: 0
Finished SIM_4239
Active: 19, Pending: 0
Finished SIM_4241
Active: 18, Pending: 0
Finished SIM_4238
Active: 17, Pending: 0
Finished SIM_4244
Active: 16, Pending: 0
Finished SIM_4243
Active: 15, Pending: 0
Finished SIM_4245
Active: 14, Pending: 0
Finished SIM_4242
Active: 13, Pending: 0
Finished SIM_4248
Active: 12, Pending: 0
Finished SIM_4247
Active: 11, Pending: 0
Finished SIM_4250
Active: 10, Pending: 0
Finished SIM_4251
Active: 9, Pending: 0
Finished SIM_4249
Active: 8, Pending: 0
Finished SIM_4252
Active: 7, Pending: 0
Finished SIM_4255
Active: 6, Pending: 0
Finished SIM_4256
Active: 5, Pending: 0
Finished SIM_4253
Active: 4, Pending: 0
Finished SIM_4254
Active: 3, Pending: 0
Finished SIM_4259
Active: 2, Pending: 0
Finished SIM_4257
Active: 1, Pending: 0
Finished SIM_4258
Active: 0, Pending: 0
Total time 0.5261521359284719 hours


Create table

In [5]:
import shutil, os, json, h5py
import pandas as pd
import numpy as np
import  scipy.stats as sps
import pycircstat

morphfiles = os.listdir(swc_path)
current_vals = pd.read_csv('../Required_files/Calibrated.csv')
current_vals = current_vals.sort_values('Target', ascending=True).reset_index(drop=True).reset_index()
current_vals = current_vals.set_index('cell_id')

def MMR (PhaseList):
    instances = np.array(np.round(PhaseList), dtype = int)
    bin_size = 36  
    gaussian = []
    for i in range(bin_size*2+1):
        gaussian.append(np.exp(-0.5*(i-bin_size)**2/(bin_size/4)**2)/(bin_size*np.sqrt(2*np.pi)/4))
    gaussian = np.array(gaussian)/gaussian[bin_size]
    waveshaped = []
    for i in instances:
        waveshape = np.zeros(360)
        indexes = np.arange(0,len(gaussian))
        indexes = indexes+i-bin_size
        for j in np.arange(0,len(gaussian)):
            if indexes[j]<0:
                indexes[j] +=360
            elif indexes[j]>359:
                indexes[j] -=360
        for j, ind in enumerate(indexes):
            waveshape[ind] = gaussian [j]
        waveshaped.append(waveshape)
    sumation = np.sum(waveshaped, axis = 0)
    sumation = sumation/np.max(sumation)
    return sumation, 1-np.mean(sumation)

all_results = []
all_traces = []
simulation_no = 0
for i in morphfiles:
    if int(i.split('rmed_')[1].split('.sw')[0]) not in range(0,360,Rotation_Steps):
        continue
    if int(i.split('_')[2]) not in current_vals.index and Only_Calibrated:
        continue
    if supra:
        frequencies = SuP_frequencies
    else:
        frequencies = Sub_frequencies
    for f in frequencies:
        if supra:
            if point:
                amplitudes = SuP_amplitudes_P
            else:
                amplitudes = SuP_amplitudes
        else:
            amplitudes = Sub_amplitudes
        for c in amplitudes:
            if allHOF:
                hof_range = 20
            else:
                hof_range = 1
            for hof in range(hof_range):
                full_path = os.getcwd().replace('\\','/') + '/Simulations/' + \
                                Selected_Sweep + '/SIM_' + str(simulation_no)
                if not Custom:    
                    if supra:
                        try:
                            full_path += '/Data.csv'
                            df = pd.read_csv(full_path)
                        except:
                            if point==0:
                                print("File missing: "+full_path)    
                            simulation_no += 1
                            continue
                        Peak_indices = np.array(df.iloc[:,0])
                        ISIs = Peak_indices[1:]-Peak_indices[:-1]
                        ISIs = 20000 / ISIs
                        mean,std=sps.norm.fit(ISIs)

                        Phases = Peak_indices % (20000/f)
                        Phases = Phases * 360 * f /20000
                        radians = np.asarray(Phases)*np.pi/180
                        mean_phase_rad = pycircstat.descriptive.mean(np.array(radians))
                        mean_phase_angle = mean_phase_rad*(180 / np.pi)
                        mean_pvalue_z=pycircstat.rayleigh(np.array(radians))
                        mean_vector_length = pycircstat.descriptive.vector_strength(np.array(radians))

                        cd, mmr = MMR(Phases)
                        Y, X = np.histogram(Phases, bins=[0, 24, 48, 72, 96, 120, 144, 168, 192, 216, 
                                                          240, 264, 288, 312, 336, 360])
                        Xp =(X[1:] + X[:-1]) / 2
                        Xp = Xp * np.pi / 180
                        normY = np.true_divide(Y, (np.max(Y)))
                        thetas = []
                        cnt = 0.0
                        for cn in range(len(cd)):
                            thetas.append(cnt)
                            cnt = cnt + 2*np.pi/len(cd)
                        all_results.append([i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''),
                                            hof, f, c, mean_vector_length, mean_pvalue_z, mean_phase_angle, mmr, 
                                            thetas, cd, Xp, normY, list(Peak_indices), list(ISIs), list(Phases)])
                    else:
                        try:
                            full_path += '/Data.csv'
                            data = pd.read_csv(full_path)
                        except:
                            if point==0:
                                print("File missing: "+full_path)    
                            simulation_no += 1
                            continue
                        row_res = [i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''),hof,f,c]
                        for r in data['0']:
                            row_res.append(r)
                        all_results.append(row_res)
                elif axon_parameters is None:
                    full_path += '/Simulation/output/v_report.h5'
                    neuron_data_output_file = full_path
                    with h5py.File(neuron_data_output_file, "r") as fi:
                        report = fi['report']
                        single_neuron = report['single_neuron']
                        data=np.array(single_neuron['data'][()])[2000:5000]
                    fi.close()
                    all_traces.append([i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''),
                                       hof,f,c,data])
                else:
                    full_path += '/Simulation/output/v_axon.h5'
                    neuron_data_output_file = full_path
                    with h5py.File(neuron_data_output_file, "r") as fi:
                        report = fi['report']
                        single_neuron = report['single_neuron']
                        data = np.array(single_neuron['data'][()])[2000:5000]
                    fi.close()
                    full_path = full_path.replace('v_axon.h5', 'v_report.h5')
                    neuron_data_output_file = full_path
                    with h5py.File(neuron_data_output_file, "r") as fi:
                        report = fi['report']
                        single_neuron = report['single_neuron']
                        soma_data=np.array(single_neuron['data'][()])[2000:5000]
                    fi.close()
                    all_traces.append([i.split('_')[0],i.split('_')[2],i.split('_')[4].replace('.swc',''),
                                       hof,f,c,soma_data,data])
                simulation_no += 1
if not Custom:
    if supra:
        gf = pd.DataFrame(all_results, columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)','HOF', 
                                                  'ES_frequency(hz)', 'ES_current(uA)', 'mean_vector_length', 
                                                  'mean_pvalue_z', 'mean_phase_angle', 'MMR', 'MMR_X', 'MMR_Y', 
                                                  'VL_X', 'VL_Y', 'Peak_indices', 'ISIs', 'Phases'])
        gf.to_pickle('../Results/Result_Tables/'+ Selected_Sweep +'.pkl')
    else:
        df = pd.DataFrame(all_results)
        df.columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)','HOF', 'ES_frequency(hz)',
                  'ES_current(uA)', 'Raw_Min(mV)', 'Raw_Max(mV)', 'Raw_Mean(mV)',
                  'Raw_Median(mV)', 'Amp_Min(mV)', 'Amp_Max(mV)', 'Amp_Mean(mV)',
                  'Amp_Median(mV)','Amp@40000(mV)', 'Phs_Min(deg)', 'Phs_Max(deg)', 
                  'Phs_Mean(deg)', 'Phs_Median(deg)', 'Phs@30000(deg)', 'Phs@40000(deg)']
        df.to_csv('../Results/Result_Tables/'+ Selected_Sweep +'.csv', index = False)
else:
    ff = pd.DataFrame(all_traces)
    if axon_parameters is None:
        ff.columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)','HOF', 'ES_frequency(hz)',
              'ES_current(uA)', 'Trace(mV)']
    else:
        ff.columns = ['Cell_Type', 'Cell_ID', 'Cell_Rotation(deg)','HOF', 'ES_frequency(hz)',
              'ES_current(uA)', 'Trace(mV)', 'Traces(mV)']
    ff.to_pickle('../Results/Result_Tables/'+ Selected_Sweep +'.pkl')    

Erase simulation files

In [6]:
import shutil, os

shutil.rmtree(os.getcwd().replace('\\','/') + '/Simulations/' + Selected_Sweep)